In [4]:
# !pip install mlxtend

In [34]:
!pip install xgboost

In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Classifiers
from sklearn.svm import NuSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier # 
import xgboost as xgb

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

# Used to ignore warnings generated from StackingCVClassifier
import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split

In [12]:
df = pd.read_csv('datasets_52721_99691_student-mat.csv')


In [17]:
X = df.drop('G3', axis=1)
y = df[['G3']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
172,GP,M,17,U,LE3,T,4,4,teacher,other,reputation,mother,1,2,0,no,yes,yes,yes,yes,yes,yes,no,4,4,4,1,3,5,0,13,11
119,GP,M,15,U,GT3,T,3,4,other,other,reputation,father,1,1,0,no,no,no,no,yes,yes,yes,no,3,4,3,1,2,4,6,14,13
389,MS,F,18,U,GT3,T,1,1,other,other,course,mother,2,2,1,no,no,no,yes,yes,yes,no,no,1,1,1,1,1,5,0,6,5
198,GP,F,17,U,GT3,T,4,4,services,teacher,home,mother,2,1,1,no,yes,no,no,yes,yes,yes,no,4,2,4,2,3,2,24,18,18
139,GP,F,15,U,GT3,T,4,4,teacher,teacher,course,mother,2,1,0,no,no,no,yes,yes,yes,yes,no,4,3,2,1,1,5,0,16,16


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

cat_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('oneHot', OneHotEncoder(handle_unknown='ignore'))])

In [19]:
num_features = df.select_dtypes(include=['int64', 'float64']).drop(['G3'], axis=1).columns
cat_features = df.select_dtypes(include=['object']).columns
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transform, num_features),
        ('cat', cat_transform, cat_features)])

In [55]:
# Initializing Support Vector classifier
# c1 = SVC(C = 50, degree = 1, gamma = "auto", kernel = "rbf", probability = True)
# c1 = SVC(random_state = 1000)

# Initializing Multi-layer perceptron  classifier
# c2 = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10),
#                             learning_rate = "constant", max_iter = 2000, random_state = 1000)

# Initialing Nu Support Vector classifier
# classifier3 = NuSVC(degree = 1, kernel = "rbf", nu = 0.25, probability = True)

# Initializing Random Forest classifier
# c3 = RandomForestClassifier(n_estimators = 500, criterion = "gini", max_depth = 10,
#                                      max_features = "auto", min_samples_leaf = 0.005,
#                                      min_samples_split = 0.005, n_jobs = -1, random_state = 1000)


c2 = xgb.XGBClassifier( objective = 'multi:softmax')

c3 = RandomForestClassifier(random_state = 1000, n_jobs = -1)
meta_c = SVC(probability = True)


In [88]:
# n_estimators = [int(x) for x in np.linspace(start = 20, stop = 500, num = 10)]
n_estimators = [5,10,20]
# Number of features to consider at every split
max_features = ['auto', 'sqrt',0.5]
# Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(5, 110, num = 11)]
max_depth = [5,10,15,20]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 3, 5,7]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
params = {'pipeline-2__rf__n_estimators': n_estimators,
               'pipeline-2__rf__max_features': max_features,
               'pipeline-2__rf__max_depth': max_depth,
               'pipeline-2__rf__min_samples_split': min_samples_split,
               'pipeline-2__rf__min_samples_leaf': min_samples_leaf,
               'pipeline-2__rf__bootstrap': bootstrap,
              "pipeline-1__xgb__n_estimators": [10, 50],
                "pipeline-1__xgb__learning_rate": [0.5, 1],}
print(params)

{'pipeline-2__rf__n_estimators': [5, 10, 20], 'pipeline-2__rf__max_features': ['auto', 'sqrt', 0.5], 'pipeline-2__rf__max_depth': [5, 10, 15, 20, None], 'pipeline-2__rf__min_samples_split': [2, 5, 10], 'pipeline-2__rf__min_samples_leaf': [1, 3, 5, 7], 'pipeline-2__rf__bootstrap': [True, False], 'pipeline-1__xgb__n_estimators': [10, 50], 'pipeline-1__xgb__learning_rate': [0.5, 1]}


In [89]:
# from sklearn.ensemble import RandomForestClassifier

# clf1 = KNeighborsClassifier(n_neighbors=1)
# clf2 = RandomForestClassifier(random_state=RANDOM_SEED)
# clf3 = GaussianNB()
# from sklearn.pipeline import make_pipeline

# pipe1 = make_pipeline(preprocessor,c2)
# pipe2 = make_pipeline(preprocessor,c3)

# pipe = Pipeline([
#     ('preprocessor', preprocessor),('rf',rf)
# ])


xgb_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('xgb', c2)])

rf_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rf', c3)])

stacking_model = StackingCVClassifier(classifiers=[xgb_pipe,rf_pipe], 
                            meta_classifier=meta_c,
                            random_state=42)

# params = {'kneighborsclassifier__n_neighbors': [1, 5],
#           'randomforestclassifier__n_estimators': [10, 50],
#           'meta_classifier__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=stacking_model, 
                    param_grid=params, 
                    cv=3,
                    refit=True)
grid.fit(X, y)

cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_[cv_keys[0]][r],
             grid.cv_results_[cv_keys[1]][r] / 2.0,
             grid.cv_results_[cv_keys[2]][r]))

print('Best parameters: %s' % grid.best_params_)
print('Accuracy: %.2f' % grid.best_score_)

KeyboardInterrupt: 

In [83]:
stacking_model.get_params()

{'classifiers': [Pipeline(steps=[('preprocessor',
                   ColumnTransformer(transformers=[('num',
                                                    Pipeline(steps=[('imputer',
                                                                     SimpleImputer(strategy='median')),
                                                                    ('scaler',
                                                                     StandardScaler())]),
                                                    Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
         'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2'],
        dtype='object')),
                                                   ('cat',
                                                    Pipeline(steps=[('imputer',
                                                                     S...
                                 interaction_constraints=None, learning_rate=None,
    

In [111]:
X

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [110]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.datasets import load_boston
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression

n_estimators = [20,50,100,250,500]
# Number of features to consider at every split
max_features = ['auto', 'sqrt',0.5]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 3, 5,7]
# Method of selecting samples for training each tree
bootstrap = [True, False]


# from sklearn.linear_model import LinearRegression

# rf = RandomForestRegressor(random_state = 1000, n_jobs = -1)


X, y = load_boston(return_X_y=True)

# X, y = load_boston(return_X_y=True)

ridge = Ridge()
lasso = Lasso()
rf = RandomForestRegressor()
lr = LinearRegression(n_jobs = -1)

from sklearn.compose import ColumnTransformer, make_column_transformer

num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = y.select_dtypes(include=['object']).columns

preprocess = make_column_transformer(
    (StandardScaler(), num_features),(OneHotEncoder(handle_unknown='ignore'),cat_features),remainder="passthrough")


stack = StackingCVRegressor(regressors=(rf, ridge),
                            meta_regressor=lr, 
                            use_features_in_secondary=True)

pipeline = make_pipeline(preprocess, stack)

params = {
#     'stackingcvregressor__lasso__alpha': [0.1, 1.0, 10.0],
          'stackingcvregressor__ridge__alpha': [0.1, 1.0, 10.0],
    'stackingcvregressor__randomforestregressor__n_estimators': n_estimators,
         }

grid = GridSearchCV(
    verbose=1,
    estimator=pipeline, 
    param_grid=params, 
    cv=5,
    refit=True
)

grid.fit(X, y)

print("Best: %f using %s" % (grid.best_score_, grid.best_params_))

AttributeError: 'numpy.ndarray' object has no attribute 'select_dtypes'

In [106]:
pipeline.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('stackingcvregressor',
   StackingCVRegressor(meta_regressor=LinearRegression(n_jobs=-1),
                       regressors=(RandomForestRegressor(), Ridge()),
                       use_features_in_secondary=True))],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'stackingcvregressor': StackingCVRegressor(meta_regressor=LinearRegression(n_jobs=-1),
                     regressors=(RandomForestRegressor(), Ridge()),
                     use_features_in_secondary=True),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'stackingcvregressor__cv': 5,
 'stackingcvregressor__meta_regressor__copy_X': True,
 'stackingcvregressor__meta_regressor__fit_intercept': True,
 'stackingcvregressor__meta_regressor__n_jobs': -1,
 'stackingcvregressor__meta_regressor__normalize': False,
 'stackingcvregressor__meta_regressor': LinearRegression(n_jobs=-1),
 'stackingcvregresso